In [13]:
import traffic_data as td
import sqlite3
import asyncio
import requests
import aiohttp
import credentials as cred

In [2]:
conn = sqlite3.connect('test2.db')

In [10]:
county = "contra costa"
cmd =\
f"""
SELECT Min_Latitude, Max_Latitude, Min_Longitude, Max_Longitude FROM counties
WHERE County LIKE '{county.upper()}'
"""
# print(cmd)
cursor = conn.cursor()
cursor.execute(cmd)
# min_lat, max_lat, min_lng, max_lng = cursor.fetchone()
bbox = cursor.fetchone()

In [11]:
bbox

(37.718629, 38.099878, -122.441584, -121.534102)

In [25]:
td.update_county_incidents(conn, "contra costa")

Page 1 processed.


In [3]:
data = td.get_county_incidents(conn, "contra costa")


            SELECT * FROM incidents
            WHERE lat BETWEEN 37.718629 AND 38.099878
            AND lng BETWEEN -122.441584 AND -121.534102
        


In [4]:
len(data)

72

In [5]:
data[data.duplicated('id')]

,id,type,severity,shortDesc,lat,lng,startTime,endTime,address,endDatetime


In [14]:
gkey = cred.google_api_key

In [15]:
data

,id,type,severity,shortDesc,lat,lng,startTime,endTime,address,endDatetime
0,2808226401283142083,4,3,Closed,37.80137,-122.27457,2023-01-10T12:18:23,2023-06-04T00:18:23,"827 Washington St, Oakland, CA 94607, USA",2023-06-04 00:18:23
1,3223177048992940125,1,1,Construction work,37.78674,-121.95176,2023-03-18T02:01:59,2023-06-07T02:01:59,"3707 Crow Canyon Rd, San Ramon, CA 94582, USA",2023-06-07 02:01:59
2,3520601258435183429,4,3,Closed,37.84354,-122.28861,2023-01-10T12:18:23,2023-06-04T03:00:00,"1327 62nd St, Emeryville, CA 94608, USA",2023-06-04 03:00:00
3,3475066287999098974,1,1,Construction work,37.78679,-122.39206,2021-08-24T14:00:00,2023-06-07T02:01:59,"399 Fremont St, San Francisco, CA 94105, USA",2023-06-07 02:01:59
4,2854691939743560747,1,1,Construction work,37.78147,-121.96436,2023-03-10T03:21:40,2023-06-07T03:21:40,"Crow Canyon Rd And Alcosta Blvd, Danville, CA ...",2023-06-07 03:21:40
...,...,...,...,...,...,...,...,...,...,...
67,1541602929065829938,1,1,Road marking operations. Two lanes closed,37.95285,-122.32823,2023-06-03T02:45:08,2023-06-03T11:00:00,"5296-5300 Riverside Ave, San Pablo, CA 94806, USA",2023-06-03 11:00:00
68,4342707188652869903,3,1,Stop and go traffic. Approach with care,37.77579,-122.40504,2023-06-03T04:38:00,2023-06-03T04:50:00,"1009 Harrison St, San Francisco, CA 94103, USA",2023-06-03 04:50:00
69,964434779545769064,3,1,Backed-up traffic. Approach with care,37.76612,-122.21851,2023-06-03T04:31:00,2023-06-03T04:50:00,"4545 Oakport St, Oakland, CA 94601, USA",2023-06-03 04:50:00
70,964434779536593932,3,1,Backed-up traffic. Approach with care,37.76383,-122.21668,2023-06-03T04:31:00,2023-06-03T04:52:00,"4911 Coliseum Wy, Oakland, CA 94601, USA",2023-06-03 04:52:00


In [18]:
slice = data.copy()[:5]
slice

,id,type,severity,shortDesc,lat,lng,startTime,endTime,address,endDatetime
0,2808226401283142083,4,3,Closed,37.80137,-122.27457,2023-01-10T12:18:23,2023-06-04T00:18:23,"827 Washington St, Oakland, CA 94607, USA",2023-06-04 00:18:23
1,3223177048992940125,1,1,Construction work,37.78674,-121.95176,2023-03-18T02:01:59,2023-06-07T02:01:59,"3707 Crow Canyon Rd, San Ramon, CA 94582, USA",2023-06-07 02:01:59
2,3520601258435183429,4,3,Closed,37.84354,-122.28861,2023-01-10T12:18:23,2023-06-04T03:00:00,"1327 62nd St, Emeryville, CA 94608, USA",2023-06-04 03:00:00
3,3475066287999098974,1,1,Construction work,37.78679,-122.39206,2021-08-24T14:00:00,2023-06-07T02:01:59,"399 Fremont St, San Francisco, CA 94105, USA",2023-06-07 02:01:59
4,2854691939743560747,1,1,Construction work,37.78147,-121.96436,2023-03-10T03:21:40,2023-06-07T03:21:40,"Crow Canyon Rd And Alcosta Blvd, Danville, CA ...",2023-06-07 03:21:40


In [ ]:
url='https://maps.googleapis.com/maps/api/geocode/json?latlng={coord}&key={google_key}'

In [24]:
addys = []
for _, row in data.iterrows():
    coord = str(row['lat']) +","+ str(row['lng'])
    try:
        response = requests.get(f"https://maps.googleapis.com/maps/api/geocode/json?latlng={coord}&key={gkey}")
        addr = response.json()['results'][0]['formatted_address']
    except:
        addr = ""
    addys.append(addr)

In [26]:
len(addys)

72

In [44]:

# def get_tasks(session):
#     tasks=[]
#     for _, row in data.iterrows():
#         coord = str(row['lat']) +","+ str(row['lng'])
#         url='https://maps.googleapis.com/maps/api/geocode/json?latlng={}&key={}'
#         print(url.format(coord, gkey))
#         tasks.append(session.get(url.format(coord, gkey)))
#     return tasks
# async def get_addresses():
#     async with aiohttp.ClientSession() as session:
#         tasks = get_tasks(session)
#         responses = await asyncio.gather(*tasks)


In [50]:
# def get_address(coord):
#     url='https://maps.googleapis.com/maps/api/geocode/json?latlng={}&key={}'
#     r = requests.get(url.format(coord,gkey))
#     return r.json()['results'][0]['formatted_address']

In [107]:
async def get_address(url, session):
    try:
        async with session.get(url=url) as response:
            resp = await response.json()
            # print("Got response")
            return resp['results'][0]
    except Exception as e:
        print("Unable to get response, error due to {}".format(e.__class__))

async def get_batch_addresses(df, addresses):
    async with aiohttp.ClientSession() as session:
        tasks=[]
        for _, row in df.iterrows():
            coord = str(row['lat']) +","+ str(row['lng'])
            url = f'https://maps.googleapis.com/maps/api/geocode/json?latlng={coord}&key={gkey}'
            tasks.append(asyncio.ensure_future(get_address(url, session)))
        addys = await asyncio.gather(*tasks)
        for addr in addys:
            addresses.append(addr)

In [84]:
testadds=[]
await get_batch_addresses(testadds)

In [97]:
print(testadds[0]['address_components'])
my_item = next((item for item in testadds[0]['address_components'] if 'administrative_area_level_2' in item['types']), None)
print(my_item)

[{'long_name': '827', 'short_name': '827', 'types': ['street_number']}, {'long_name': 'Washington Street', 'short_name': 'Washington St', 'types': ['route']}, {'long_name': 'Downtown Oakland', 'short_name': 'Downtown Oakland', 'types': ['neighborhood', 'political']}, {'long_name': 'Oakland', 'short_name': 'Oakland', 'types': ['locality', 'political']}, {'long_name': 'Alameda County', 'short_name': 'Alameda County', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'California', 'short_name': 'CA', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'United States', 'short_name': 'US', 'types': ['country', 'political']}, {'long_name': '94607', 'short_name': '94607', 'types': ['postal_code']}, {'long_name': '4029', 'short_name': '4029', 'types': ['postal_code_suffix']}]
{'long_name': 'Alameda County', 'short_name': 'Alameda County', 'types': ['administrative_area_level_2', 'political']}


In [120]:
testadds[0]['formatted_address']

'827 Washington St, Oakland, CA 94607, USA'

In [86]:
data['geocode'] = testadds

In [121]:
def get_county(geocode):
    info = geocode['address_components']
    county_dict = next((item for item in info if 'administrative_area_level_2' in item['types']), None)
    return county_dict['long_name']

def get_formatted_address(geocode):
    return geocode['formatted_address']

In [108]:
slice_geos = []
await get_batch_addresses(slice, slice_geos)
slice_geos

[{'address_components': [{'long_name': '827',
    'short_name': '827',
    'types': ['street_number']},
   {'long_name': 'Washington Street',
    'short_name': 'Washington St',
    'types': ['route']},
   {'long_name': 'Downtown Oakland',
    'short_name': 'Downtown Oakland',
    'types': ['neighborhood', 'political']},
   {'long_name': 'Oakland',
    'short_name': 'Oakland',
    'types': ['locality', 'political']},
   {'long_name': 'Alameda County',
    'short_name': 'Alameda County',
    'types': ['administrative_area_level_2', 'political']},
   {'long_name': 'California',
    'short_name': 'CA',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'United States',
    'short_name': 'US',
    'types': ['country', 'political']},
   {'long_name': '94607', 'short_name': '94607', 'types': ['postal_code']},
   {'long_name': '4029',
    'short_name': '4029',
    'types': ['postal_code_suffix']}],
  'formatted_address': '827 Washington St, Oakland, CA 94607, USA',
  

In [109]:
len(slice_geos)

5

In [111]:
slice['geocode'] = slice_geos
slice

,id,type,severity,shortDesc,lat,lng,startTime,endTime,address,endDatetime,geocode
0,2808226401283142083,4,3,Closed,37.80137,-122.27457,2023-01-10T12:18:23,2023-06-04T00:18:23,"827 Washington St, Oakland, CA 94607, USA",2023-06-04 00:18:23,"{'address_components': [{'long_name': '827', '..."
1,3223177048992940125,1,1,Construction work,37.78674,-121.95176,2023-03-18T02:01:59,2023-06-07T02:01:59,"3707 Crow Canyon Rd, San Ramon, CA 94582, USA",2023-06-07 02:01:59,"{'address_components': [{'long_name': '3707', ..."
2,3520601258435183429,4,3,Closed,37.84354,-122.28861,2023-01-10T12:18:23,2023-06-04T03:00:00,"1327 62nd St, Emeryville, CA 94608, USA",2023-06-04 03:00:00,"{'address_components': [{'long_name': '1327', ..."
3,3475066287999098974,1,1,Construction work,37.78679,-122.39206,2021-08-24T14:00:00,2023-06-07T02:01:59,"399 Fremont St, San Francisco, CA 94105, USA",2023-06-07 02:01:59,"{'address_components': [{'long_name': '399', '..."
4,2854691939743560747,1,1,Construction work,37.78147,-121.96436,2023-03-10T03:21:40,2023-06-07T03:21:40,"Crow Canyon Rd And Alcosta Blvd, Danville, CA ...",2023-06-07 03:21:40,{'address_components': [{'long_name': 'Crow Ca...


In [113]:
slice['county'] = slice['geocode'].apply(get_county)


In [124]:
geocodes = []
await get_batch_addresses(data, geocodes)
data['geocode'] = geocodes
data['county'] = data['geocode'].apply(get_county)
data['address'] = data['geocode'].apply(get_formatted_address)

In [125]:
data

,id,type,severity,shortDesc,lat,lng,startTime,endTime,endDatetime,geocode,county,address
0,2808226401283142083,4,3,Closed,37.80137,-122.27457,2023-01-10T12:18:23,2023-06-04T00:18:23,2023-06-04 00:18:23,"{'address_components': [{'long_name': '827', '...",Alameda County,"827 Washington St, Oakland, CA 94607, USA"
1,3223177048992940125,1,1,Construction work,37.78674,-121.95176,2023-03-18T02:01:59,2023-06-07T02:01:59,2023-06-07 02:01:59,"{'address_components': [{'long_name': '3707', ...",Contra Costa County,"3707 Crow Canyon Rd, San Ramon, CA 94582, USA"
2,3520601258435183429,4,3,Closed,37.84354,-122.28861,2023-01-10T12:18:23,2023-06-04T03:00:00,2023-06-04 03:00:00,"{'address_components': [{'long_name': '1327', ...",Alameda County,"1327 62nd St, Emeryville, CA 94608, USA"
3,3475066287999098974,1,1,Construction work,37.78679,-122.39206,2021-08-24T14:00:00,2023-06-07T02:01:59,2023-06-07 02:01:59,"{'address_components': [{'long_name': '399', '...",San Francisco County,"399 Fremont St, San Francisco, CA 94105, USA"
4,2854691939743560747,1,1,Construction work,37.78147,-121.96436,2023-03-10T03:21:40,2023-06-07T03:21:40,2023-06-07 03:21:40,{'address_components': [{'long_name': 'Crow Ca...,Contra Costa County,"Crow Canyon Rd And Alcosta Blvd, Danville, CA ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
67,1541602929065829938,1,1,Road marking operations. Two lanes closed,37.95285,-122.32823,2023-06-03T02:45:08,2023-06-03T11:00:00,2023-06-03 11:00:00,{'address_components': [{'long_name': '5296-53...,Contra Costa County,"5296-5300 Riverside Ave, San Pablo, CA 94806, USA"
68,4342707188652869903,3,1,Stop and go traffic. Approach with care,37.77579,-122.40504,2023-06-03T04:38:00,2023-06-03T04:50:00,2023-06-03 04:50:00,"{'address_components': [{'long_name': '1009', ...",San Francisco County,"1009 Harrison St, San Francisco, CA 94103, USA"
69,964434779545769064,3,1,Backed-up traffic. Approach with care,37.76612,-122.21851,2023-06-03T04:31:00,2023-06-03T04:50:00,2023-06-03 04:50:00,"{'address_components': [{'long_name': '4545', ...",Alameda County,"4545 Oakport St, Oakland, CA 94601, USA"
70,964434779536593932,3,1,Backed-up traffic. Approach with care,37.76383,-122.21668,2023-06-03T04:31:00,2023-06-03T04:52:00,2023-06-03 04:52:00,"{'address_components': [{'long_name': '4911', ...",Alameda County,"4911 Coliseum Wy, Oakland, CA 94601, USA"
